In [ ]:
def get_technicals(symbol):
  import requests, pandas, datetime
  from pandas.tseries.offsets import BDay

  '''
  Get technical data from price/volume information using Yahoo Finance API.
  '''
  try:
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

    querystring = {"interval":"1d","symbol": symbol,"range":"1y","region":"US"}

    headers = {
      "X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81",
      "X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring).json()

    date = response['chart']['result'][0]['timestamp']
    prices = response['chart']['result'][0]['indicators']['quote'][0]
    prices, date = pandas.DataFrame(prices), pandas.DataFrame(date, columns = ['date'])
    prices['date'] = date
    prices['date'] = prices.date.apply(lambda a: datetime.datetime.fromtimestamp(a))
    prx = prices.close.iloc[-1]

    prices['o_n_gap_per'] = prices.high / prices.close.shift(periods = 1) - 1
    prices['o_n_gap_ind'] = prices.o_n_gap_per.apply(lambda a: 1 if a > 1 else 0)

    # Recent Pump in Last 60 Days Trading
    recent_pump = len(prices[(prices.o_n_gap_ind == 1) & (prices.date >= prices.date.max() - BDay(60))]) # Last 60 trading days

    prices['dollar_vol'] = prices['close'] * prices['volume']

    # Recent Spiker with history of overnight gap ups with high volume
    o_n_gaps = len(prices[(prices.o_n_gap_ind == 1) & (prices.dollar_vol >= 1000000) & (prices.date < prices.date.max() - BDay(60))])

    # Was the stock trading above $10/share in the past 12 months
    above_10 = 1 if len(prices[prices.high > 10]) > 0 else 0

    # Is the stock net accumulated (green vol > red vol) ?
    prices['prev_close'] = prices.close.shift(periods = 1)
    prices['perc_change'] = (prices.close / prices.prev_close) - 1
    prices['green_red'] = prices.perc_change.apply(lambda a: 'G' if a > 0 else 'R')
    accum = 1 if (prices.volume[prices.green_red == 'G'].sum() - prices.volume[prices.green_red == 'R'].sum()) > 0 else 0

    # Low Near 52wk Low
    low52 = prices.low.min()
    low = prices.low.iloc[-1].min()
    near_52wl = 1 if (low/low52)-1 <= 0.10 else 0

    return recent_pump, o_n_gaps, above_10, accum, near_52wl
  except:
    return -1, -1, -1, -1, -1

In [ ]:
'''
stocks = ['WORX',
'TMPO',
'MEDS',
'BDTX',
'TIVC',
'NERV',
'BXRX',
'IMPL',
'ALLR',
'GDTC',
'HCTI',
'GLMD',
'HCTI',
'JZXN',
'AIMD',
'PMN',
'BJDX',
'BFRG',
'ABVC',
'GBNH',
'BRSH',
'MSGM',
'RENB',
'DCTH',
'NVOS',
'EBET',
'ADTX',
'GDC',
'NAOV',
'MINM',
'ACER',
'EDBL',
'HSDT',
'FWBI',
'PALI',
'NBSE',
'AREB',
'WHLR',
'SLE',
'ASTI',
'COSM',
'LIFW',
'AWIN']
'''

In [ ]:
def get_pump_stats(symbol):
  import requests, pandas, datetime

  '''
  Get technical data from price/volume information using Yahoo Finance API.
  '''

  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

  querystring = {"interval":"1d","symbol": symbol,"range":"1y","region":"US"}

  headers = {
    "X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81",
    "X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring).json()

  date = response['chart']['result'][0]['timestamp']
  prices = response['chart']['result'][0]['indicators']['quote'][0]
  prices, date = pandas.DataFrame(prices), pandas.DataFrame(date, columns = ['date'])
  prices['date'] = date
  prices['date'] = prices.date.apply(lambda a: datetime.datetime.fromtimestamp(a))

  prx = prices.close.iloc[-1]

  # determine whether pre-market data is included
  prices['o_n_gap_per'] = prices.high / prices.close.shift(periods = 1) - 1
  prices['o_n_gap_ind'] = prices.o_n_gap_per.apply(lambda a: 1 if a > 0.5 else 0)
  o_n_gaps = len(prices[prices.o_n_gap_ind == 1])
  prices['dollar_vol'] = prices['close'] * prices['volume']

  o_n_gap_data = prices[prices.o_n_gap_ind == 1]
  hist_of_vol = len(o_n_gap_data[o_n_gap_data.dollar_vol >= 10000000])
  dates = o_n_gap_data.date[o_n_gap_data.dollar_vol >= 10000000]

  if hist_of_vol > 0:
    return 1, dates
  else:
    return 0, []